# Práctica sobre Padrón #

La siguiente sección de la práctica se abordará si ya se tienen suficientes conocimientos de Spark, en concreto de el manejo de DataFrames, y el manejo de tablas de Hive a través de Spark.sql

6.1) Comenzamos realizando la misma práctica que hicimos en Hive en Spark, importando el csv. Sería recomendable intentarlo con opciones que quiten las "" de los campos, que ignoren los espacios innecesarios en los campos, que sustituyan los valores vacíos por 0 y que infiera el esquema

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = (SparkSession
        .builder
        .appName("padron")
        .getOrCreate())

fich= "/FileStore/tables/Rango_Edades_Seccion_202205.csv"

df=spark.read.format("csv").option("header","true").option("inferschema","true").option("emptyValue", 0).option("delimiter",";").load(fich)

df_bien=df.na.fill(value=0).withColumn("DESC_DISTRITO",trim(col("desc_distrito"))).withColumn("DESC_BARRIO",trim(col("desc_barrio")))


6.3) Enumera todos los barrios diferentes

In [0]:
df_bien.select('DESC_BARRIO').distinct().show()
df_bien.select(countDistinct("desc_barrio")).show()

+---------------+
|    DESC_BARRIO|
+---------------+
|  LOS JERONIMOS|
|         CORTES|
|      TRAFALGAR|
|  VALDEACEDERAS|
|    PROSPERIDAD|
|          IBIZA|
|    EMBAJADORES|
|         LA PAZ|
|        ACACIAS|
|PALOS DE MOGUER|
|     GUINDALERA|
|   LOS CARMENES|
|       IMPERIAL|
|         ALUCHE|
|        PALACIO|
|  CASA DE CAMPO|
|  BELLAS VISTAS|
|    LAS AGUILAS|
|       JUSTICIA|
|    UNIVERSIDAD|
+---------------+
only showing top 20 rows

+---------------------------+
|count(DISTINCT desc_barrio)|
+---------------------------+
|                        131|
+---------------------------+



6.4) Crea una vista temporal de nombre "padron" y a través de ella cuenta el número de barrios diferentes que hay.

In [0]:
df_bien.createOrReplaceTempView("tabla")
spark.sql("""SELECT DISTINCT(DESC_BARRIO) 
                FROM tabla""").show()
spark.sql("""SELECT count(distinct(DESC_BARRIO)) FROM tabla""").show()

+---------------+
|    DESC_BARRIO|
+---------------+
|  LOS JERONIMOS|
|         CORTES|
|      TRAFALGAR|
|  VALDEACEDERAS|
|    PROSPERIDAD|
|          IBIZA|
|    EMBAJADORES|
|         LA PAZ|
|        ACACIAS|
|PALOS DE MOGUER|
|     GUINDALERA|
|   LOS CARMENES|
|       IMPERIAL|
|         ALUCHE|
|        PALACIO|
|  CASA DE CAMPO|
|  BELLAS VISTAS|
|    LAS AGUILAS|
|       JUSTICIA|
|    UNIVERSIDAD|
+---------------+
only showing top 20 rows

+---------------------------+
|count(DISTINCT DESC_BARRIO)|
+---------------------------+
|                        131|
+---------------------------+



6.5) Crea una nueva columna que muestre la longitud de los campos de la columna DESC_DISTRITO y que se llame "longitud".

In [0]:
from pyspark.sql.functions import *

df2=df_bien.withColumn("longitud", length(col("DESC_DISTRITO")))

6.6) Crea una nueva columna que muestre el valor 5 para cada uno de los registros de la tabla.

In [0]:
df3=df2.withColumn("valor5",lit(5))

6.7) Borra esta columna.

In [0]:
df4=df3.drop(col("valor5"))

6.8) Particiona el DataFrame por las variables DESC_DISTRITO y DESC_BARRIO

In [0]:
padron_particionado=df4.repartition(col("DESC_DISTRITO"),col("DESC_BARRIO"))

6.9) Almacénalo en caché. Consulta en el puerto 4040 (UI de Spark) de tu usuario local el estado
de los rdds almacenados.

In [0]:
padron_particionado.cache()

Out[78]: DataFrame[COD_DISTRITO: int, DESC_DISTRITO: string, COD_DIST_BARRIO: int, DESC_BARRIO: string, COD_BARRIO: int, COD_DIST_SECCION: int, COD_SECCION: int, COD_EDAD_INT: int, EspanolesHombres: int, EspanolesMujeres: int, ExtranjerosHombres: int, ExtranjerosMujeres: int, longitud: int]

6.10) Lanza una consulta contra el DF resultante en la que muestre el número total de 
"espanoleshombres", "espanolesmujeres", extranjeroshombres" y "extranjerosmujeres" 
para cada barrio de cada distrito. Las columnas distrito y barrio deben ser las primeras en 
aparecer en el show. Los resultados deben estar ordenados en orden de más a menos 
según la columna "extranjerosmujeres" y desempatarán por la columna 
"extranjeroshombres".

In [0]:
padron_particionado.groupBy(col("DESC_BARRIO"),col("DESC_DISTRITO")).agg(count("espanoleshombres").alias("numespanoleshombres"), count('espanolesmujeres').alias("numespanolesmujeres"), count('extranjeroshombres').alias("numextranjeroshombres"), count('extranjerosmujeres').alias("numextranjerosmujeres")).show(10)

+-------------+-------------------+-------------------+-------------------+---------------------+---------------------+
|  DESC_BARRIO|      DESC_DISTRITO|numespanoleshombres|numespanolesmujeres|numextranjeroshombres|numextranjerosmujeres|
+-------------+-------------------+-------------------+-------------------+---------------------+---------------------+
|   MIRASIERRA|FUENCARRAL-EL PARDO|               1991|               1991|                 1991|                 1991|
|   CASTELLANA|          SALAMANCA|               1485|               1485|                 1485|                 1485|
|  EL SALVADOR|SAN BLAS-CANILLEJAS|                780|                780|                  780|                  780|
|CASA DE CAMPO|    MONCLOA-ARAVACA|               1096|               1096|                 1096|                 1096|
| VALDEFUENTES|          HORTALEZA|               2542|               2542|                 2542|                 2542|
|   MARROQUINA|          MORATALAZ|     

6.11) Elimina el registro en caché

In [0]:
spark.catalog.clearCache()

6.12) Crea un nuevo DataFrame a partir del original que muestre únicamente una columna con 
DESC_BARRIO, otra con DESC_DISTRITO y otra con el número total de "espanoleshombres" 
residentes en cada distrito de cada barrio. Únelo (con un join) con el DataFrame original a 
través de las columnas en común.

In [0]:
df_nuevo=df_bien.groupBy("DESC_BARRIO","DESC_DISTRITO").agg(sum("espanoleshombres").alias("numero_espanoleshombres"))
df_nuevo.show(10)

df_joined=df_nuevo.join(df_bien, on=["DESC_BARRIO","DESC_DISTRITO"], how='right')

df_joined.show(3)

+-----------+-------------+-----------------------+
|DESC_BARRIO|DESC_DISTRITO|numero_espanoleshombres|
+-----------+-------------+-----------------------+
|    ACACIAS|   ARGANZUELA|                  15372|
|   CANILLAS|    HORTALEZA|                  16716|
|PROSPERIDAD|    CHAMARTIN|                  14398|
|    LEGAZPI|   ARGANZUELA|                   8829|
| SAN ISIDRO|  CARABANCHEL|                  14456|
|       GOYA|    SALAMANCA|                  10799|
|UNIVERSIDAD|       CENTRO|                  12228|
| NI�O JESUS|       RETIRO|                   6534|
|        SOL|       CENTRO|                   2878|
|  TRAFALGAR|     CHAMBERI|                   9353|
+-----------+-------------+-----------------------+
only showing top 10 rows

+-----------+-------------+-----------------------+------------+---------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|DESC_BARRIO|DESC_DISTRITO|numero_espano

6.13) Repite la función anterior utilizando funciones de ventana. (over(Window.partitionBy.....)).

_Nota: Las funciones de ventana pyspark operan en un grupo de filas (como frame, partition) y devuelven un único valor para cada fila de entrada. Pyspark soporta 3 tipos de funciones ventana: ranking, analíticas y de agregación_.

In [0]:
from pyspark.sql.window import Window

windowSpecAgg= Window.partitionBy("DESC_BARRIO","DESC_DISTRITO")

df_ventana=df_bien.withColumn("sumespanoleshombres",sum(col("espanoleshombres")).over(windowSpecAgg))

df_ventana.show(3)


+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+-------------------+
|COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|sumespanoleshombres|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+-------------------+
|           2|   ARGANZUELA|            202|    ACACIAS|         2|            2015|         15|           0|               5|               6|                 0|                 0|              15372|
|           2|   ARGANZUELA|            202|    ACACIAS|         2|            2015|         15|           1|               2|               9|                 1|                 1|           

6.14) Mediante una función Pivot muestra una tabla (que va a ser una tabla de contingencia) que
contenga los valores totales (la suma de valores) de espanolesmujeres para cada distrito y 
en cada rango de edad (COD_EDAD_INT). Los distritos incluidos deben ser únicamente 
CENTRO, BARAJAS y RETIRO y deben figurar como columnas

_Nota: El pivot de pyspark es usado para la rotación de data de una columna de un DataFrame a varias columnas. Es una función de agregación. Mejora el performance del data y es menos costoso en análisis de datos_.

In [0]:
dist=("CENTRO","BARAJAS","RETIRO")

df_pivot=padron_particionado.groupBy("COD_EDAD_INT").pivot("DESC_DISTRITO",dist).sum('EspanolesMujeres').orderBy(col("COD_EDAD_INT"))

display(df_pivot)

COD_EDAD_INT,CENTRO,BARAJAS,RETIRO
0,256,144,303
1,237,151,315
2,191,182,379
3,218,170,373
4,229,196,414
5,227,225,413
6,233,243,403
7,231,230,464
8,234,241,410
9,237,225,425


6.15) Utilizando este nuevo DF, crea 3 columnas nuevas que hagan referencia a qué porcentaje 
de la suma de "espanolesmujeres" en los tres distritos para cada rango de edad representa 
cada uno de los tres distritos. Debe estar redondeada a 2 decimales. Puedes imponerte la 
condición extra de no apoyarte en ninguna columna auxiliar creada para el caso.

In [0]:
sumcols=col("CENTRO")+ col("BARAJAS")+ col("RETIRO")

df_3col=df_pivot.withColumn("pctgCentro",round(col("CENTRO")/(sumcols) *100,2)).withColumn("pctgBarajas",round(col("BARAJAS")/(sumcols) *100,2)).withColumn("pctgRetiro",round(col("RETIRO")/(sumcols) *100,2))

display(df_3col)

COD_EDAD_INT,CENTRO,BARAJAS,RETIRO,pctgCentro,pctgBarajas,pctgRetiro
0,256,144,303,36.42,20.48,43.1
1,237,151,315,33.71,21.48,44.81
2,191,182,379,25.4,24.2,50.4
3,218,170,373,28.65,22.34,49.01
4,229,196,414,27.29,23.36,49.34
5,227,225,413,26.24,26.01,47.75
6,233,243,403,26.51,27.65,45.85
7,231,230,464,24.97,24.86,50.16
8,234,241,410,26.44,27.23,46.33
9,237,225,425,26.72,25.37,47.91


6.16) Guarda el archivo csv original particionado por distrito y por barrio (en ese orden) en un 
directorio local. Consulta el directorio para ver la estructura de los ficheros y comprueba 
que es la esperada.

6.17) Haz el mismo guardado pero en formato parquet. Compara el peso del archivo con el 
resultado anterior

In [0]:
df_bien.write.format("csv").option("header","true").mode("overwrite").partitionBy("DESC_DISTRITO","DESC_BARRIO").save("/FileStore/shared_uploads/irene.castro@bosonit.com")

# df.write.format("parquet").option("header","true").mode("overwrite").partitionBy("DESC_DISTRITO","DESC_BARRIO").save("/FileStore/shared_uploads/irene.castro@bosonit.com") #